In [71]:
#Setting up the code and installing required packages 
%load_ext autotime
from util import *
from glob import glob
import matplotlib.pyplot as plt
from shapely import wkt
from shapely.geometry import LineString, Polygon
import geopandas as gpd
from shapely.geometry import Point, LineString
import numpy as np
from shapelysmooth import taubin_smooth
pd.set_option("display.max_columns", None)

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime


In [83]:
#Navigate to intersects shapefile and separates out the year, finds how many years from 1800 and how many years to 2100
gdf = gpd.read_file(f"Data/Merged Intersects_UniqueID/WaihekeIsland_Intersects.shp")
gdf["Date"] = pd.to_datetime(gdf.ShorelineI, dayfirst=True, format='mixed')
gdf["Year"] = gdf.Date.dt.year
gdf["YearsSinceBase"] = (gdf.Date - pd.Timestamp(1800, 1, 1)).dt.days / 365.25
gdf["YearsUntilFuture"] = (
    pd.Timestamp(2100, 1, 1) - gdf.Date
    ).dt.days / 365.25
gdf.Date = gdf.Date.astype(str)
gdf["TransectID"] = gdf.Unique_ID.astype(np.int64)
gdf

,Join_Count,TARGET_FID,TransectID,TransOrder,ShorelineI,BaselineID,Distance,IntersectX,IntersectY,Uncertaint,Unique_ID,Date,geometry,Year,YearsSinceBase,YearsUntilFuture
0,1,0,101001256584,0.0,01/02/2004,0.0,-22.99,1.782700e+06,5.927888e+06,5.11,1.010013e+11,2004-02-01,POINT Z (1782699.654 5927888.083 0.000),2004,204.079398,95.915127
1,1,1,101001256584,0.0,11/02/1961,0.0,-21.64,1.782698e+06,5.927888e+06,3.72,1.010013e+11,1961-02-11,POINT Z (1782698.329 5927887.815 0.000),1961,161.108830,138.885695
2,1,2,101001256584,0.0,01/03/2011,0.0,-25.00,1.782702e+06,5.927888e+06,0.88,1.010013e+11,2011-03-01,POINT Z (1782701.620 5927888.481 0.000),2011,211.156742,88.837782
3,1,3,101001256584,0.0,10/21/1987,0.0,-22.74,1.782699e+06,5.927888e+06,4.29,1.010013e+11,1987-10-21,POINT Z (1782699.405 5927888.033 0.000),1987,187.797399,112.197125
4,1,4,101001256584,0.0,10/24/1980,0.0,-21.89,1.782699e+06,5.927888e+06,4.29,1.010013e+11,1980-10-24,POINT Z (1782698.577 5927887.865 0.000),1980,180.807666,119.186858
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4109,1,4109,101001853562,0.0,04/25/2017,2.0,-34.87,1.784397e+06,5.926929e+06,0.73,1.010019e+11,2017-04-25,POINT Z (1784397.176 5926929.454 0.000),2017,217.308693,82.685832
4110,1,4110,101001853562,0.0,11/01/2010,2.0,-34.14,1.784398e+06,5.926930e+06,0.88,1.010019e+11,2010-01-11,POINT Z (1784397.577 5926930.063 0.000),2010,210.023272,89.971253
4111,1,4111,101001853562,0.0,10/24/1980,2.0,-37.20,1.784396e+06,5.926928e+06,4.29,1.010019e+11,1980-10-24,POINT Z (1784395.895 5926927.512 0.000),1980,180.807666,119.186858
4112,1,4112,101001853562,0.0,11/02/1961,2.0,-36.94,1.784396e+06,5.926928e+06,3.72,1.010019e+11,1961-02-11,POINT Z (1784396.037 5926927.727 0.000),1961,161.108830,138.885695


In [3]:
# Filter to just Karekare for testing
#gdf = gdf.cx[1727077.5508:1735465.0761,5897869.0262:5910640.2418]
#gdf

,ShorelineI,BaselineID,Distance,IntersectX,IntersectY,Uncertaint,Unique_ID,Date,geometry,Year,YearsSinceBase,YearsUntilFuture,TransectID
0,01/03/2011,0.0,-37.16,1.730591e+06,5.908710e+06,8.59,1.006327e+11,2011-03-01,POINT Z (1730590.982 5908710.362 0.000),2011,211.156742,88.837782,100632729920
1,08/19/1960,0.0,-51.70,1.730600e+06,5.908699e+06,3.76,1.006327e+11,1960-08-19,POINT Z (1730600.339 5908699.234 0.000),1960,160.626968,139.367556,100632729920
2,01/04/2017,0.0,-32.09,1.730588e+06,5.908714e+06,2.94,1.006327e+11,2017-04-01,POINT Z (1730587.720 5908714.241 0.000),2017,217.242984,82.751540,100632729920
3,01/02/2004,0.0,-48.27,1.730598e+06,5.908702e+06,10.05,1.006327e+11,2004-02-01,POINT Z (1730598.132 5908701.858 0.000),2004,204.079398,95.915127,100632729920
4,01/04/2022,0.0,-23.18,1.730582e+06,5.908721e+06,2.07,1.006327e+11,2022-04-01,POINT Z (1730581.983 5908721.063 0.000),2022,222.242300,77.752225,100632729920
...,...,...,...,...,...,...,...,...,...,...,...,...,...
85101,04/05/2016,2.0,-288.76,1.731087e+06,5.904261e+06,8.84,1.006321e+11,2016-05-04,POINT Z (1731087.039 5904261.021 0.000),2016,216.334018,83.660507,100632146004
85102,04/07/2010,2.0,-284.87,1.731083e+06,5.904261e+06,8.84,1.006321e+11,2010-07-04,POINT Z (1731083.142 5904260.921 0.000),2010,210.499658,89.494867,100632146004
85103,04/30/2024,2.0,-347.31,1.731146e+06,5.904263e+06,8.83,1.006321e+11,2024-04-30,POINT Z (1731145.565 5904262.521 0.000),2024,224.323066,75.671458,100632146004
85104,05/18/2020,2.0,-297.29,1.731096e+06,5.904261e+06,8.84,1.006321e+11,2020-05-18,POINT Z (1731095.558 5904261.239 0.000),2020,220.372348,79.622177,100632146004


In [84]:
def get_transects(intersects):
  p1 = intersects.geometry[intersects.Distance.idxmin()].coords[0]
  p2 = intersects.geometry[intersects.Distance.idxmax()].coords[0]
  azimuth = math.degrees(math.atan2(p1[0]-p2[0], p1[1]-p2[1]))
  if azimuth < 0:
      azimuth += 360
  return pd.Series({"Azimuth": azimuth, "geometry": LineString([p1, p2])})

lines = gdf.groupby("TransectID")[["geometry", "Distance"]].apply(get_transects)
lines.crs = gdf.crs
lines

,Azimuth,geometry
TransectID,,
101000646283,37.184706,"LINESTRING Z (1779460.160 5928020.514 0.000, 1..."
101000647289,37.184707,"LINESTRING Z (1779453.750 5928028.610 0.000, 1..."
101000648298,37.184705,"LINESTRING Z (1779446.240 5928035.256 0.000, 1..."
101000650261,342.072081,"LINESTRING Z (1779407.480 5928041.725 0.000, 1..."
101000651277,342.072078,"LINESTRING Z (1779398.024 5928038.465 0.000, 1..."
...,...,...
101011753654,308.379225,"LINESTRING Z (1779139.485 5926446.757 0.000, 1..."
101011754687,308.379225,"LINESTRING Z (1779134.113 5926438.256 0.000, 1..."
101011757298,272.361375,"LINESTRING Z (1779124.656 5926409.511 0.000, 1..."


In [85]:
lines["dist_to_neighbour"] = lines.distance(lines.shift(-1))
breakpoints = lines.dist_to_neighbour[lines.dist_to_neighbour > 15]
lines["group"] = pd.Series(range(len(breakpoints)), index=breakpoints.index)
lines["group"] = lines.group.bfill().fillna(len(breakpoints)).astype(int)
transect_metadata = lines[["Azimuth", "group"]].to_dict(orient="index")

In [86]:
transect_metadata

{101000646283: {'Azimuth': 37.18470637691588, 'group': 0},
 101000647289: {'Azimuth': 37.18470678514857, 'group': 0},
 101000648298: {'Azimuth': 37.184704654597695, 'group': 0},
 101000650261: {'Azimuth': 342.0720810204791, 'group': 1},
 101000651277: {'Azimuth': 342.0720784831029, 'group': 1},
 101000652279: {'Azimuth': 342.0720782788596, 'group': 1},
 101000653771: {'Azimuth': 321.3401917452891, 'group': 2},
 101000654777: {'Azimuth': 321.3401917466474, 'group': 2},
 101000655779: {'Azimuth': 321.34019174800994, 'group': 2},
 101000657422: {'Azimuth': 300.7246932899666, 'group': 3},
 101000658456: {'Azimuth': 300.72469328820273, 'group': 3},
 101000659476: {'Azimuth': 300.724693863921, 'group': 3},
 101000660488: {'Azimuth': 300.7246932870748, 'group': 3},
 101000661493: {'Azimuth': 300.7246932883655, 'group': 3},
 101000662495: {'Azimuth': 300.7246932884002, 'group': 3},
 101000663495: {'Azimuth': 300.72469385958414, 'group': 3},
 101000664498: {'Azimuth': 300.72469328583765, 'group

In [87]:
#Linear regression is run here. See util.py for the breakdown on linear_models
linear_models = fit(gdf, transect_metadata)
linear_models

,TransectID,slope,intercept,group,r2_score,mae,mse,rmse
0,101000646283,0.106055,-45.520155,0,0.734818,1.129870,1.734423,1.316975
1,101000647289,0.071362,-41.200458,0,0.388919,1.560543,3.003707,1.733121
2,101000648298,0.082961,-44.826049,0,0.395734,1.854435,3.945099,1.986227
3,101000650261,0.008719,-30.965070,1,0.004469,2.082160,6.357569,2.521422
4,101000651277,0.009953,-30.878887,1,0.004685,2.360437,7.900769,2.810831
...,...,...,...,...,...,...,...,...
571,101011753654,-0.099187,-11.380524,43,0.368011,2.172993,6.342262,2.518385
572,101011754687,-0.081011,-14.030067,43,0.370638,1.628269,4.183291,2.045310
573,101011757298,-0.104969,-10.775308,44,0.523244,1.649200,3.768793,1.941338
574,101011758298,-0.084843,-13.879784,44,0.378775,1.897093,5.243523,2.289874


In [8]:
#Only run if rolling average is needed. Otherwise SKIP THIS
#linear_models = fit(gdf, transect_metadata)
#rolled_slopes = linear_models.groupby("group").slope.rolling(10, min_periods=1).mean().dropna().reset_index(level=0)
#linear_models.slope = rolled_slopes.slope
#linear_models.dropna(inplace=True)
#linear_models

time: 361 µs (started: 2024-08-07 16:42:36 +12:00)


In [88]:
#Coordinates of the projected shoreline are plotted here

#Changed coordinate function by making old_x and old_y negative 
def calculate_new_coordinates(old_x, old_y, bearing, distance):
    bearing_radians = math.radians(bearing)
    new_x = old_x + (distance * math.sin(bearing_radians))
    new_y = old_y + (distance * math.cos(bearing_radians))
    point = Point(new_x, new_y)
    assert not point.is_empty
    return point

#Removed other model equations and changed Azimuth addtion from 180 to 360 deg
def predict(
    df: pd.DataFrame,
    linear_models: pd.DataFrame,
    transect_metadata: dict,
):
    """_summary_

    Args:
        df (pd.DataFrame): dataframe with columns: TransectID, Date, Distance, YearsSinceBase
        linear_models (pd.DataFrame): dataframe with columns: TransectID, slope, intercept
        transect_metadata (dict): dict lookup of TransectID to Azimuth & group
        
    Returns:
        pd.DataFrame: resulting prediction points for the year 2100
    """
    results = []
    for i, row in linear_models.iterrows():
        transect_ID = row.TransectID
        transect_df = df[df.TransectID == transect_ID]
        latest_row = transect_df[transect_df.Date == transect_df["Date"].max()].iloc[0]
        future_year = int(row.get("FUTURE_YEAR", FUTURE_YEAR))
        result = row.to_dict()
        result.update({
            "TransectID": transect_ID,
            "BaselineID": latest_row.BaselineID,
            "group": row.group,
            "Year": future_year,
            "ocean_point": calculate_new_coordinates(
                latest_row.geometry.x,
                latest_row.geometry.y,
                transect_metadata[transect_ID]["Azimuth"] + 180,
                500,
            ),
        })
        
        model = "linear"
        slope = row.slope
        intercept = row.intercept

        predicted_distance = slope * (future_year - 1800) + intercept
        distance_difference = latest_row.Distance - predicted_distance
        result[f"{model}_model_point"] = calculate_new_coordinates(
            latest_row.geometry.x,
            latest_row.geometry.y,
            transect_metadata[transect_ID]["Azimuth"],
            distance_difference,
        )
        result[f"{model}_model_predicted_distance"] = predicted_distance
        result[f"{model}_model_distance"] = distance_difference
        results.append(result)
    results = gpd.GeoDataFrame(results)
    return results

In [89]:
#Projection file is created here with the stats and coordinate points in table format
results = predict(gdf, linear_models, transect_metadata)
results

,TransectID,slope,intercept,group,r2_score,mae,mse,rmse,BaselineID,Year,ocean_point,linear_model_point,linear_model_predicted_distance,linear_model_distance
0,1.010006e+11,0.106055,-45.520155,0.0,0.734818,1.129870,1.734423,1.316975,4.0,2100,POINT (1779154.1245450925 5927617.103046248),POINT (1779450.6386966005 5928007.962610681),-13.703548,-9.396452
1,1.010006e+11,0.071362,-41.200458,0.0,0.388919,1.560543,3.003707,1.733121,4.0,2100,POINT (1779148.6385283498 5927626.417188603),POINT (1779446.3544096262 5928018.860845564),-19.791890,-7.408110
2,1.010006e+11,0.082961,-44.826049,0.0,0.395734,1.854435,3.945099,1.986227,4.0,2100,POINT (1779142.0233869734 5927634.242904833),POINT (1779438.4735822347 5928025.018187699),-19.937747,-9.502253
3,1.010007e+11,0.008719,-30.965070,1.0,0.004469,2.082160,6.357569,2.521422,4.0,2100,POINT (1779561.5358263077 5927565.551680209),POINT (1779408.9094871345 5928037.30584147),-28.349394,-4.170606
4,1.010007e+11,0.009953,-30.878887,1.0,0.004685,2.360437,7.900769,2.810831,4.0,2100,POINT (1779552.0798424843 5927562.292845871),POINT (1779399.5354758091 5928033.793566286),-27.893026,-4.436974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,1.010118e+11,-0.099187,-11.380524,43.0,0.368011,2.172993,6.342262,2.518385,5.0,2100,POINT (1779533.8324563708 5926134.43459647),POINT (1779133.7073155998 5926451.333719642),-41.136675,10.416675
572,1.010118e+11,-0.081011,-14.030067,43.0,0.370638,1.628269,4.183291,2.045310,5.0,2100,POINT (1779527.8345015713 5926126.428550819),POINT (1779129.6953578866 5926441.754764255),-38.333253,7.883253
573,1.010118e+11,-0.104969,-10.775308,44.0,0.523244,1.649200,3.768793,1.941338,5.0,2100,POINT (1779624.2309808887 5926388.910404921),POINT (1779116.7461437446 5926409.8376137875),-42.266143,7.916143
574,1.010118e+11,-0.084843,-13.879784,44.0,0.378775,1.897093,5.243523,2.289874,5.0,2100,POINT (1779625.6257533398 5926378.844388716),POINT (1779119.263357353 5926399.72531236),-39.332748,6.792748


In [90]:
#Spatial reference added to the results
results.set_geometry("linear_model_point", inplace=True, crs=2193)
results

,TransectID,slope,intercept,group,r2_score,mae,mse,rmse,BaselineID,Year,ocean_point,linear_model_point,linear_model_predicted_distance,linear_model_distance
0,1.010006e+11,0.106055,-45.520155,0.0,0.734818,1.129870,1.734423,1.316975,4.0,2100,POINT (1779154.1245450925 5927617.103046248),POINT (1779450.639 5928007.963),-13.703548,-9.396452
1,1.010006e+11,0.071362,-41.200458,0.0,0.388919,1.560543,3.003707,1.733121,4.0,2100,POINT (1779148.6385283498 5927626.417188603),POINT (1779446.354 5928018.861),-19.791890,-7.408110
2,1.010006e+11,0.082961,-44.826049,0.0,0.395734,1.854435,3.945099,1.986227,4.0,2100,POINT (1779142.0233869734 5927634.242904833),POINT (1779438.474 5928025.018),-19.937747,-9.502253
3,1.010007e+11,0.008719,-30.965070,1.0,0.004469,2.082160,6.357569,2.521422,4.0,2100,POINT (1779561.5358263077 5927565.551680209),POINT (1779408.909 5928037.306),-28.349394,-4.170606
4,1.010007e+11,0.009953,-30.878887,1.0,0.004685,2.360437,7.900769,2.810831,4.0,2100,POINT (1779552.0798424843 5927562.292845871),POINT (1779399.535 5928033.794),-27.893026,-4.436974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,1.010118e+11,-0.099187,-11.380524,43.0,0.368011,2.172993,6.342262,2.518385,5.0,2100,POINT (1779533.8324563708 5926134.43459647),POINT (1779133.707 5926451.334),-41.136675,10.416675
572,1.010118e+11,-0.081011,-14.030067,43.0,0.370638,1.628269,4.183291,2.045310,5.0,2100,POINT (1779527.8345015713 5926126.428550819),POINT (1779129.695 5926441.755),-38.333253,7.883253
573,1.010118e+11,-0.104969,-10.775308,44.0,0.523244,1.649200,3.768793,1.941338,5.0,2100,POINT (1779624.2309808887 5926388.910404921),POINT (1779116.746 5926409.838),-42.266143,7.916143
574,1.010118e+11,-0.084843,-13.879784,44.0,0.378775,1.897093,5.243523,2.289874,5.0,2100,POINT (1779625.6257533398 5926378.844388716),POINT (1779119.263 5926399.725),-39.332748,6.792748


In [91]:
#Line and polygon shapefiles are created here 
def prediction_results_to_line_polygon(results: gpd.GeoDataFrame):
    lines = []
    polygons = []
    for group_name, group_data in results.groupby(["BaselineID", "group"]):
        if len(group_data) > 1:
            # Convert the points to LineString
            line = LineString(list(group_data.geometry))
            lines.append(line)
            # Convert the points to a closed Polygon
            polygon = Polygon(list(group_data.geometry) + list(group_data.ocean_point)[::-1])
            polygons.append(polygon)
    lines = gpd.GeoSeries(lines, crs=2193)
    polygons = gpd.GeoSeries(polygons, crs=2193)
    return lines, polygons
lines, poly = prediction_results_to_line_polygon(results)

In [92]:
m = lines.explore()
lines.apply(lambda line: taubin_smooth(line, steps=500)).to_file("Projections/WaihekeIsland_output_line_smoothed.shp")
lines.apply(lambda line: taubin_smooth(line, steps=500)).explore(m=m, color="red")

In [15]:
#Saving line and polygon projection file to Z drive. Change file location accordingly  
lines, poly = prediction_results_to_line_polygon(results)
lines.to_file("Z:\Lalita\RNC Cont\......\BigBay_projection_output_lines.shp")
poly.to_file("Z:\Lalita\RNC Cont\.......\BigBay_projection_output_polygon.shp")

DriverIOError: Failed to create file Z:\Lalita\RNC Cont\....../BigBay_projection_output_lines.shp: No such file or directory

time: 859 ms (started: 2024-08-07 16:42:39 +12:00)


In [70]:
lines, poly = prediction_results_to_line_polygon(results)
lines.to_file("Projections/WaihekeIsland_projection_output_line.shp")
poly.to_file("Projections/WaihekeIsland_projection_output_polygon.shp")

In [ ]:
#Saving line and polygon projection file to folder in VS Code. Change file location accordingly
lines, poly = prediction_results_to_line_polygon(results)
lines.to_file("Projections\NorthIsland_projection_output_lines.shp")
poly.to_file("Projections\NorthIsland_projection_output_polygon.shp")

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 11-12: malformed \N character escape (3394349576.py, line 3)

In [ ]:
#Quick visualisation of projected polygon and historic shorelines 
m = poly.explore(tiles="Esri.WorldImagery")
gpd.GeoDataFrame(results.drop(columns=["ocean_point", "linear_model_point"]), geometry=results.linear_model_point).explore(m=m)
gdf.explore("Year", legend=True, m=m)